In [4]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/Colab Notebooks/ch06'
import os
os.chdir(dir_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from common import config
config.GPU = True
from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity, to_gpu
from dataset import ptb
from better_rnnlm import BetterRnnlm


# ハイパーパラメータの設定
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

if config.GPU:
    corpus = to_gpu(corpus)
    corpus_val = to_gpu(corpus_val)
    corpus_test = to_gpu(corpus_test)

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size,
                time_size=time_size, max_grad=max_grad)

    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print('valid perplexity: ', ppl)

    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr

    model.reset_state()
    print('-' * 50)


# テストデータでの評価
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('test perplexity: ', ppl_test)


| epoch 1 |  iter 1 / 1327 | time 13[s] | perplexity 9999.86
| epoch 1 |  iter 21 / 1327 | time 19[s] | perplexity 3674.17
| epoch 1 |  iter 41 / 1327 | time 24[s] | perplexity 2110.66
| epoch 1 |  iter 61 / 1327 | time 30[s] | perplexity 1245.52
| epoch 1 |  iter 81 / 1327 | time 36[s] | perplexity 1038.83
| epoch 1 |  iter 101 / 1327 | time 41[s] | perplexity 876.30
| epoch 1 |  iter 121 / 1327 | time 47[s] | perplexity 818.34
| epoch 1 |  iter 141 / 1327 | time 52[s] | perplexity 746.39
| epoch 1 |  iter 161 / 1327 | time 58[s] | perplexity 701.69
| epoch 1 |  iter 181 / 1327 | time 63[s] | perplexity 694.23
| epoch 1 |  iter 201 / 1327 | time 69[s] | perplexity 596.95
| epoch 1 |  iter 221 / 1327 | time 74[s] | perplexity 569.69
| epoch 1 |  iter 241 / 1327 | time 80[s] | perplexity 538.59
| epoch 1 |  iter 261 / 1327 | time 85[s] | perplexity 532.02
| epoch 1 |  iter 281 / 1327 | time 91[s] | perplexity 520.53
| epoch 1 |  iter 301 / 1327 | time 96[s] | perplexity 453.42
| epoch 1

改善前のRNNLMではパープレキシティは144であったので、かなり精度が上がっていることがわかる。

感想  
単純なRNNからかなり精度が上がったことを確認できた。またRNNの構造を理解することがきた。GPUを使っても計算に４時間近くかかった。次回の文章生成は気になっていたところなので楽しみである。

参考  
ゼロから始めるDeepLearning２　第六章「ゲート付きRNN」